In [40]:
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import netCDF4 as nc
import numpy as np
from pyproj import Proj, Transformer, CRS
from pyproj.aoi import AreaOfInterest
from pyproj.database import query_utm_crs_info
import plotly as py
from plotly.graph_objs import *
from scipy.io import netcdf
from mpl_toolkits.basemap import Basemap
import plotly.express as px
import plotly.graph_objects as go
from metpy.calc import relative_humidity_from_specific_humidity
from metpy.units import units

from utils import GeoTsObject

In [3]:
geo_ts = GeoTsObject()

In [4]:
dataset = geo_ts.read_netcdf(as_dataframe=False)


In [24]:
# Save data as datasets
temp = dataset['temperature']
psurf = dataset['pressure']
wind = dataset['wind_speed']
qair = dataset['specific_humidity']
rainfall = dataset['rainfall_flux']
snowfall = dataset['snowfall_flux']
radiation = dataset['shortwave_down']

In [48]:
# Rasample to daily for efficiency
temp_m = temp.resample(time = 'M').mean()
psurf_m = psurf.resample(time = 'M').mean()
wind_m = wind.resample(time = 'M').mean()
qair_m = qair.resample(time = 'M').mean()
rainfall_m = rainfall.resample(time = 'M').mean()
snowfall_m = snowfall.resample(time = 'M').mean()
radiation_m = radiation.resample(time = 'M').mean()

In [56]:
# Conversions
temp_m = temp_m.update(
    {'temp_degc': (('time', 'lat', 'lon'), temp_m.Tair.data - 273.15)})
precip_m = rainfall_m.update(
    {'precip': (('time', 'lat', 'lon'), (rainfall_m.Rainf.data + snowfall_m.Snowf.data) * 3600)})
psurf_m = psurf_m.update(
    {'psurf_hpa': (('time', 'lat', 'lon'), psurf_m.PSurf.data * 0.01)})

rel_hum_values = relative_humidity_from_specific_humidity(psurf_m.psurf_hpa.values * 0.01* units.hPa, 
                                                            temp_m.temp_degc.values * units.degC, 
                                                            qair_m.Qair.values).to('percent').magnitude

In [62]:
temp_m.temp_degc.values

array([[[       nan,        nan,        nan, ..., -1067.6   ,
         -1068.1001, -1068.4995],
        [       nan,        nan,        nan, ..., -1068.4001,
         -1068.8999, -1068.9001],
        [       nan,        nan,        nan, ..., -1068.5002,
         -1068.7002, -1068.7998],
        ...,
        [-1090.3   , -1090.3   , -1090.3997, ..., -1099.2999,
         -1099.5002, -1099.5   ],
        [-1090.7999, -1091.1   , -1091.1001, ..., -1100.2998,
         -1100.4001, -1099.9999],
        [-1091.3999, -1091.4998, -1091.5001, ..., -1100.5001,
         -1100.7003, -1100.1997]],

       [[       nan,        nan,        nan, ..., -1066.0002,
         -1066.2999, -1066.7998],
        [       nan,        nan,        nan, ..., -1066.6   ,
         -1067.2   , -1067.1998],
        [       nan,        nan,        nan, ..., -1066.9   ,
         -1066.9999, -1067.3002],
        ...,
        [-1087.    , -1086.8994, -1086.7997, ..., -1096.4995,
         -1096.7003, -1096.8002],
        [-10